## Merge Leah's 2019-2022 data with CBC_Colony

In [1]:
library("tidyverse")
library("vegan")
#library("phyloseq")
library("dplyr")
library("ggplot2")
library("stringr")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: permute

Loading required package: lattice

This is vegan 2.6-6.1



In [2]:
getwd()

[1] "/Users/brookesienkiewicz/Documents/sctld/Demographics"

In [76]:
setwd('/Users//brookesienkiewicz/Documents/sctld/SCTLD_samples/Sample_Data/')

In [77]:
colony_data <- read.csv('CBC_ColonyData.csv')

In [78]:
leah_colony <- read.csv('/Users//brookesienkiewicz/Documents/sctld/Demographics/CBC_Fate_2019to2022.csv')

In [79]:
colony_data <- colony_data[, -1]

In [80]:
# extract year from Date_InitialTag and add it to new column
colony_data$Year_tagged <- format(as.Date(colony_data$Date_InitialTag, format="%m/%d/%y"), "%Y")

In [81]:
head(colony_data)

,Date_InitialTag,Transect,TransectNum,OldTagNum,NewTagNum,Species,Meter,Meters_90,Direction,Size_Class,⋯,X122023_Condition,X122023_Percentage,X012024_Condition,X012024_Percentage,X022024_Condition,X022024_Percentage,X042024_Condition,X042024_Percentage,COLONIES_TO_LOOK_AT,Year_tagged
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,6/21/19,CBC30N,1,349,1,SSID,1.5,0.50,right,5,⋯,Diseased,,Diseased,,Diseased,,,,,2019
2,6/21/19,CBC30N,1,334,2,PAST,1.4,0.10,left,4,⋯,CLB,100%,Dead,,TL,90%,,,,2019
3,6/21/19,CBC30N,1,346,3,SSID,6.3,0.80,right,5,⋯,CLP,80%,CLP,,CLB,98%,,,,2019
4,10/15/19,CBC30N,1,407,4,PSTR,6.5,0.50,right,3,⋯,Dead,,Dead,,Dead,,,,,2019
5,6/24/19,CBC30N,1,394,5,SSID,4.3,1.20,left,5,⋯,Diseased,,Diseased,,Diseased,,,,,2019
6,10/15/19,CBC30N,1,413,6,PSTR,8.3,1.75,left,NA,⋯,Dead,,Dead,,Dead,,,,,2019


In [82]:
colnames(colony_data)

[1] "Date_InitialTag"          "Transect"                
 [3] "TransectNum"              "OldTagNum"               
 [5] "NewTagNum"                "Species"                 
 [7] "Meter"                    "Meters_90"               
 [9] "Direction"                "Size_Class"              
[11] "Notes_June_2019"          "MaxDiameter"             
[13] "Height"                   "Date_DocumentedDisease"  
[15] "Date_DocumentedMortality" "Notes_May2022"           
[17] "Sept_2023"                "Nov_2023"                
[19] "Dec_2023"                 "Jan_2024"                
[21] "Feb_2024"                 "April_2024"              
[23] "X062019_Condition"        "X062019_Percentage"      
[25] "X052022_Condition"        "X122022_Condition"       
[27] "X052023_Condition"        "X092023_Condition"       
[29] "X092023_Percentage"       "X112023_Condition"       
[31] "X112023_Percentage"       "X122023_Condition"       
[33] "X122023_Percentage"       "X012024_Condition"       
[35] "X012024_Percentage"       "X022024_Condition"       
[37] "X022024_Percentage"       "X042024_Condition"       
[39] "X042024_Percentage"       "COLONIES_TO_LOOK_AT"     
[41] "Year_tagged"

In [83]:
# make col that combines species, tag, and transect 
# first match transect names
leah_colony$Transect %>% unique()

[1] "CBC Lagoon" "CBC30N"     "SR30N"

In [84]:
# replace CBC Lagoon to Lagoon 
leah_colony$Transect[leah_colony$Transect == "CBC Lagoon"] <- "Lagoon"

In [85]:
# make col that combines species, tag, and transect 
# first match transect names
leah_colony$Transect %>% unique()

[1] "Lagoon" "CBC30N" "SR30N"

In [86]:
# look at Leah's data 
head(leah_colony)

,X,Species,Date_Initial,Max_Diameter,Height,New_Tag,Tag_Orig,Transect,Time_Point,Condition
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,PAST,2019-06-23,30,10,10,361,Lagoon,May22,Healthy
2,2,MCAV,2019-06-23,27,34,14,369,Lagoon,May22,Healthy
3,3,MCAV,2019-06-23,29,24,15,379,Lagoon,May22,Diseased
4,4,MCAV,2019-06-23,29,14,17,301,Lagoon,May22,Diseased
5,5,PAST,2019-06-23,18,13,18,365,Lagoon,May22,Diseased
6,6,SSID,2019-06-23,32,27,19,300,Lagoon,May22,Diseased


In [87]:
# nas in tag num?
sum(is.na(leah_colony$New_Tag))

[1] 20

In [88]:
# replace new tag num with old tag num where that are nas
leah_colony$New_Tag <- ifelse(is.na(leah_colony$New_Tag), leah_colony$Tag_Orig, leah_colony$New_Tag)

In [89]:
# make col that combines species, tag, and transect in leah's data
leah_colony$ID <- paste(leah_colony$Species, leah_colony$New_Tag, leah_colony$Transect, sep = "_")
head(leah_colony$ID)

[1] "PAST_10_Lagoon" "MCAV_14_Lagoon" "MCAV_15_Lagoon" "MCAV_17_Lagoon"
[5] "PAST_18_Lagoon" "SSID_19_Lagoon"

In [90]:
# make col that combines species, tag, and transect in colony data
colony_data$ID <- paste(colony_data$Species, colony_data$NewTagNum, colony_data$Transect, sep = "_")
head(colony_data$ID)

[1] "SSID_1_CBC30N" "PAST_2_CBC30N" "SSID_3_CBC30N" "PSTR_4_CBC30N"
[5] "SSID_5_CBC30N" "PSTR_6_CBC30N"

In [91]:
# separate 2019 and 5/22, 12/22 conditions in leah's data
wide_data <- pivot_wider(
  data = leah_colony,
  id_cols = ID,
  names_from = Time_Point,
  values_from = Condition,
  names_prefix = "lh_Condition_"
)
head(wide_data)

ID,lh_Condition_May22,lh_Condition_Dec22,lh_Condition_Oct19
<chr>,<chr>,<chr>,<chr>
PAST_10_Lagoon,Healthy,Healthy,Healthy
MCAV_14_Lagoon,Healthy,Healthy,Healthy
MCAV_15_Lagoon,Diseased,Diseased,Healthy
MCAV_17_Lagoon,Diseased,Dead,Healthy
PAST_18_Lagoon,Diseased,Diseased,Healthy
SSID_19_Lagoon,Diseased,Diseased,Healthy


In [44]:
# Change Lagoon 
#colony_data$Transect[colony_data$Transect == "Lagoon"] <- "LAGOON"

In [92]:
# get subset of colonies tagged in 2019 (to be able to track since the beginning)
colonies_2019 <- subset(colony_data, colony_data$Year_tagged=="2019")
head(colonies_2019)

,Date_InitialTag,Transect,TransectNum,OldTagNum,NewTagNum,Species,Meter,Meters_90,Direction,Size_Class,⋯,X122023_Percentage,X012024_Condition,X012024_Percentage,X022024_Condition,X022024_Percentage,X042024_Condition,X042024_Percentage,COLONIES_TO_LOOK_AT,Year_tagged,ID
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,6/21/19,CBC30N,1,349,1,SSID,1.5,0.50,right,5,⋯,,Diseased,,Diseased,,,,,2019,SSID_1_CBC30N
2,6/21/19,CBC30N,1,334,2,PAST,1.4,0.10,left,4,⋯,100%,Dead,,TL,90%,,,,2019,PAST_2_CBC30N
3,6/21/19,CBC30N,1,346,3,SSID,6.3,0.80,right,5,⋯,80%,CLP,,CLB,98%,,,,2019,SSID_3_CBC30N
4,10/15/19,CBC30N,1,407,4,PSTR,6.5,0.50,right,3,⋯,,Dead,,Dead,,,,,2019,PSTR_4_CBC30N
5,6/24/19,CBC30N,1,394,5,SSID,4.3,1.20,left,5,⋯,,Diseased,,Diseased,,,,,2019,SSID_5_CBC30N
6,10/15/19,CBC30N,1,413,6,PSTR,8.3,1.75,left,NA,⋯,,Dead,,Dead,,,,,2019,PSTR_6_CBC30N


In [93]:
colonies_2019$Transect %>% unique()

[1] "CBC30N" "SR30N"  "Lagoon"

In [94]:
# pull rows that are present in both dfs
matching_rows <- merge(wide_data, colonies_2019, by = 'ID')
head(matching_rows)

,ID,lh_Condition_May22,lh_Condition_Dec22,lh_Condition_Oct19,Date_InitialTag,Transect,TransectNum,OldTagNum,NewTagNum,Species,⋯,X122023_Condition,X122023_Percentage,X012024_Condition,X012024_Percentage,X022024_Condition,X022024_Percentage,X042024_Condition,X042024_Percentage,COLONIES_TO_LOOK_AT,Year_tagged
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,CNAT_64_SR30N,Healthy,Healthy,Healthy,6/21/19,SR30N,2,354,64,CNAT,⋯,CLP,,Dead,,CLP,,,,,2019
2,MCAV_12_Lagoon,Dead,Dead,Healthy,6/23/19,Lagoon,3,304,12,MCAV,⋯,Dead,,Dead,,Dead,,,,,2019
3,MCAV_14_CBC30N,Diseased,Dead,Healthy,6/24/19,CBC30N,1,381,14,MCAV,⋯,Dead,,Dead,,Dead,,,,,2019
4,MCAV_14_Lagoon,Healthy,Healthy,Healthy,6/23/19,Lagoon,3,369,14,MCAV,⋯,Healthy,,Healthy,,Healthy,,,,,2019
5,MCAV_15_CBC30N,Diseased,Dead,Healthy,6/26/19,CBC30N,1,359,15,MCAV,⋯,Dead,,Dead,,Dead,,,,,2019
6,MCAV_15_Lagoon,Diseased,Diseased,Healthy,6/23/19,Lagoon,3,379,15,MCAV,⋯,Dead,,Dead,,Dead,,,,,2019


In [95]:
# look at leah's condition ("Condition_date") vs colony_data condition ("X052022_Condition")
match_conditions<-matching_rows[, c("ID", "Transect","lh_Condition_Oct19","lh_Condition_May22","lh_Condition_Dec22","X062019_Condition","X052022_Condition","X122022_Condition")]

In [96]:
nas <- is.na(match_conditions)

In [97]:
rows_with_na <- match_conditions[apply(nas, 1, any), ]

# Print the rows with NA values
rows_with_na

,ID,Transect,lh_Condition_Oct19,lh_Condition_May22,lh_Condition_Dec22,X062019_Condition,X052022_Condition,X122022_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
11,MCAV_24_CBC30N,CBC30N,NA,Healthy,Healthy,Healthy,Healthy,Healthy


In [98]:
# replace na with healthy 
row_index <- which(match_conditions$ID == "MCAV_24_CBC30N")

# Replace the entry in lh_Condition_Oct19 at the identified row index
match_conditions$lh_Condition_Oct19[row_index] <- "Healthy"

In [99]:
colnames(match_conditions)

[1] "ID"                 "Transect"           "lh_Condition_Oct19"
[4] "lh_Condition_May22" "lh_Condition_Dec22" "X062019_Condition" 
[7] "X052022_Condition"  "X122022_Condition"

In [100]:
# for each year, find rows where they don't match (mismatched conditions)
mismatched_conditions19 <- match_conditions[match_conditions$lh_Condition_Oct19 != match_conditions$X062019_Condition, ]
mismatched_conditions19

ID,Transect,lh_Condition_Oct19,lh_Condition_May22,lh_Condition_Dec22,X062019_Condition,X052022_Condition,X122022_Condition
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [101]:
# for each year, find rows where they don't match (mismatched conditions)
mismatched_conditions522 <- match_conditions[match_conditions$lh_Condition_May22 != match_conditions$X052022_Condition, ]
mismatched_conditions522

,ID,Transect,lh_Condition_Oct19,lh_Condition_May22,lh_Condition_Dec22,X062019_Condition,X052022_Condition,X122022_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
16,MCAV_61_SR30N,SR30N,Healthy,Dead,Dead,Healthy,Healthy,Dead
18,MCAV_7_CBC30N,CBC30N,Healthy,Increased Old Mortality,Healthy,Healthy,Healthy,Healthy
27,PAST_13_Lagoon,Lagoon,Healthy,Increased Old Mortality,Increased Old Mortality,Healthy,Healthy,Healthy
33,PAST_24_Lagoon,Lagoon,Healthy,Healthy,Healthy,Healthy,Dead,Dead
35,PAST_56_SR30N,SR30N,Healthy,Diseased,Healthy,Healthy,Healthy,Healthy
39,PAST_68_SR30N,SR30N,Healthy,Diseased,Healthy,Healthy,Healthy,Healthy
50,PSTR_4_Lagoon,Lagoon,Healthy,Diseased,Healthy,Healthy,Healthy,Healthy
54,PSTR_62_SR30N,SR30N,Healthy,Increased Old Mortality,Dead,Healthy,Healthy,Dead
58,SSID_10_CBC30N,CBC30N,Healthy,Diseased,Diseased,Healthy,Healthy,Diseased


In [102]:
# sort by transect 
mismatched_conditions522[(mismatched_conditions522$Transect=="SR30N"),]


,ID,Transect,lh_Condition_Oct19,lh_Condition_May22,lh_Condition_Dec22,X062019_Condition,X052022_Condition,X122022_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
16,MCAV_61_SR30N,SR30N,Healthy,Dead,Dead,Healthy,Healthy,Dead
35,PAST_56_SR30N,SR30N,Healthy,Diseased,Healthy,Healthy,Healthy,Healthy
39,PAST_68_SR30N,SR30N,Healthy,Diseased,Healthy,Healthy,Healthy,Healthy
54,PSTR_62_SR30N,SR30N,Healthy,Increased Old Mortality,Dead,Healthy,Healthy,Dead


In [143]:
# unclear if mcav 61 was diseased in 05/22 or 12/22
# notes for 5/22 "Damage, mostly dead" and "Moved in storm? Found upside down and righted colony, few polyps left" 
# datasheets: sampled as healthy in 5/22, labelled as dead in 12/22  
## LEAVE AS HEALTHY  in 5/22, dead after

# past 56 sampled as diseased 5/22, but healthy in 12/22
# looks diseased in 5/22 https://drive.google.com/drive/u/0/folders/1OhCgbgrVgvgabie2Zx2IW6kfoEHOpYgd
# pic in 12/22 doesn't look healthy (but is NOT diseased) but sampled as healthy https://drive.google.com/drive/u/0/folders/1Y4SxDofp0PDFFKvjuFr6wN3znsbBzS6J
## CHANGE 5/22 to diseased, leave as healthy after
colony_data[colony_data$ID=="PAST_56_SR30N",c("X052022_Condition")]<- "Diseased"

# past 68 sampled as healthy in 5/22, healthy in 12/22, 
## LEAVE AS HEALTHY 

# past 62 (406) sampled as healthy in 5/22 but labelled as inc old mort & ' one spot of focal CL', dead in 12/22  
# https://drive.google.com/drive/u/0/folders/1OhCgbgrVgvgabie2Zx2IW6kfoEHOpYgd
## LEAVE AS healthy 5/22 & dead in 12/22

# add to list of colonies to double check at the end 
colonies_to_check <- list('MCAV_61_SR30N', 'PAST_56_SR30N', 'PAST_68_SR30N', 'PSTR_62_SR30N')

In [104]:
# sort by transect 
mismatched_conditions522[(mismatched_conditions522$Transect=="CBC30N"),]

,ID,Transect,lh_Condition_Oct19,lh_Condition_May22,lh_Condition_Dec22,X062019_Condition,X052022_Condition,X122022_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
18,MCAV_7_CBC30N,CBC30N,Healthy,Increased Old Mortality,Healthy,Healthy,Healthy,Healthy
58,SSID_10_CBC30N,CBC30N,Healthy,Diseased,Diseased,Healthy,Healthy,Diseased


In [145]:
# mcav 7 - sampled as healthy 5/22, increased partial old mortality 
## LEAVE as healthy

# ssid 10 - 5/22 diseased or healthy?
# sampled as diseased tissue 
## CHANGE TO diseased 5/22
colony_data[colony_data$ID=="SSID_10_CBC30N",c("X052022_Condition")]<- "Diseased"

colonies_to_check <- append(colonies_to_check, c("MCAV_7_CBC30N","SSID_10_CBC30N"))

In [106]:
# sort by transect 
mismatched_conditions522[(mismatched_conditions522$Transect=="Lagoon"),]

,ID,Transect,lh_Condition_Oct19,lh_Condition_May22,lh_Condition_Dec22,X062019_Condition,X052022_Condition,X122022_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
27,PAST_13_Lagoon,Lagoon,Healthy,Increased Old Mortality,Increased Old Mortality,Healthy,Healthy,Healthy
33,PAST_24_Lagoon,Lagoon,Healthy,Healthy,Healthy,Healthy,Dead,Dead
50,PSTR_4_Lagoon,Lagoon,Healthy,Diseased,Healthy,Healthy,Healthy,Healthy
64,SSID_26_Lagoon,Lagoon,Healthy,Diseased,Diseased,Healthy,Dead,Dead


In [146]:
# past 13 
## LEAVE as healthy 

# past 24 
# NOT sampled in 5/22 (presumed dead?) https://drive.google.com/drive/u/0/folders/1neqfKWg7dLPKk1f4FS6QuN_CXcLy9eOe
# sampled in 12/22 as healthy (to me - this looks like a different coral) https://drive.google.com/drive/u/0/folders/1WPO_drh8oriwWwarB7gQ4g5aGgrd7Qyu
# I don't even see it in 2019 photos: https://drive.google.com/drive/u/0/folders/1LeYcwxCL0uu-b3GyRJiC6dyUuUP3hDC7
# never sampled again after that (guessing it was presumed dead?)
## CHANGE to healthy 5/22 and 12/22, not visited after that 
colony_data[colony_data$ID=="PAST_24_Lagoon",c("X052022_Condition")]<- "Healthy"
colony_data[colony_data$ID=="PAST_24_Lagoon",c("X122022_Condition")]<- "Healthy"

# pstr 4
# sampled as healthy 2019 - 2022, leah's notes say ' TL (F, SA) - sampled as healthy!!' with condition as 'Diseased/Arrested TL' in 5/22
## LEAVE as healthy 

# ssid 26
# sampled as diseased in 12/22, no sample in 5/22 
# looks alive but diseased in all 2022, never sampled after (guessing it was presumed dead?)
## CHANGE to diseased in 5/22 https://drive.google.com/drive/u/0/folders/1neqfKWg7dLPKk1f4FS6QuN_CXcLy9eOe
colony_data[colony_data$ID=="SSID_26_Lagoon",c("X052022_Condition")]<- "Diseased"

colonies_to_check <- append(colonies_to_check, c("PAST_13_Lagoon","PAST_24_Lagoon",'PSTR_4_Lagoon', 'SSID_26_Lagoon'))

In [126]:
# add not visited to the rest of PAST_24_Lagoon condition dates 
condition_columns<-colnames(colony_data)[grepl("Condition",colnames(colony_data))]
condition_columns

[1] "X062019_Condition" "X052022_Condition" "X122022_Condition"
 [4] "X052023_Condition" "X092023_Condition" "X112023_Condition"
 [7] "X122023_Condition" "X012024_Condition" "X022024_Condition"
[10] "X042024_Condition"

In [127]:
filter_condition_columns <- condition_columns[!condition_columns %in% c('X062019_Condition', 'X052022_Condition','X122022_Condition')]
filter_condition_columns

[1] "X052023_Condition" "X092023_Condition" "X112023_Condition"
[4] "X122023_Condition" "X012024_Condition" "X022024_Condition"
[7] "X042024_Condition"

In [117]:
colony_data[colony_data$ID=="PAST_24_Lagoon",filter_condition_columns]<- "Not_visited"

In [120]:
# for each year, find rows where they don't match (mismatched conditions)
mismatched_conditions1222 <- match_conditions[match_conditions$lh_Condition_Dec22 != match_conditions$X122022_Condition, ]
mismatched_conditions1222

,ID,Transect,lh_Condition_Oct19,lh_Condition_May22,lh_Condition_Dec22,X062019_Condition,X052022_Condition,X122022_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
14,MCAV_59_SR30N,SR30N,Healthy,Diseased,Increased Old Mortality,Healthy,Diseased,Diseased
27,PAST_13_Lagoon,Lagoon,Healthy,Increased Old Mortality,Increased Old Mortality,Healthy,Healthy,Healthy
29,PAST_19_CBC30N,CBC30N,Healthy,Diseased,Dead,Healthy,Diseased,Diseased
33,PAST_24_Lagoon,Lagoon,Healthy,Healthy,Healthy,Healthy,Dead,Dead
64,SSID_26_Lagoon,Lagoon,Healthy,Diseased,Diseased,Healthy,Dead,Dead
66,SSID_3_CBC30N,CBC30N,Healthy,Diseased,Healthy,Healthy,Diseased,Diseased
68,SSID_33_Lagoon,Lagoon,Healthy,Healthy,Healthy,Healthy,Healthy,Diseased
69,SSID_36_Lagoon,Lagoon,Healthy,Diseased,Healthy,Healthy,Diseased,Diseased
73,SSID_51_SR30N,SR30N,Healthy,Diseased,Not Found,Healthy,Diseased,Diseased


In [121]:
# sort by transect 
mismatched_conditions1222[(mismatched_conditions1222$Transect=="SR30N"),]

,ID,Transect,lh_Condition_Oct19,lh_Condition_May22,lh_Condition_Dec22,X062019_Condition,X052022_Condition,X122022_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
14,MCAV_59_SR30N,SR30N,Healthy,Diseased,Increased Old Mortality,Healthy,Diseased,Diseased
73,SSID_51_SR30N,SR30N,Healthy,Diseased,Not Found,Healthy,Diseased,Diseased
80,SSID_74_SR30N,SR30N,Healthy,Healthy,Diseased,Healthy,Healthy,Healthy


In [147]:
# mcav 59 
# not sampled after 2019 until 2024 - notes in 5/22 say missed sample
# looks like sctld in 5/22 https://drive.google.com/drive/u/0/folders/1OhCgbgrVgvgabie2Zx2IW6kfoEHOpYgd
# doesn't look like TL in 12/22 but no sample anyway https://drive.google.com/drive/u/0/folders/1Y4SxDofp0PDFFKvjuFr6wN3znsbBzS6J
# maybe looks a little pale in 2023 but stayed pretty much the same since 12/22 https://drive.google.com/drive/u/0/folders/1048Vv_PCukpgXwjENUzXZeehxhV3oyh7
## CHANGE to diseased 5/22
colony_data[colony_data$ID=="MCAV_59_SR30N",c("X052022_Condition")]<- "Diseased"
## CHANGE to healthy for 12/22
colony_data[colony_data$ID=="MCAV_59_SR30N",c("X122022_Condition")]<- "Healthy"
colony_data[colony_data$ID=="MCAV_59_SR30N",c("X052023_Condition")]<- "Healthy"
## UPDATE status 9/23 pale 
colony_data[colony_data$ID=="MCAV_59_SR30N",filter_condition_columns]
colony_data[colony_data$ID=="MCAV_59_SR30N",c("X092023_Condition")]<- "CLP"
colony_data[colony_data$ID=="MCAV_59_SR30N",c("X112023_Condition")]<- "CLP"
colony_data[colony_data$ID=="MCAV_59_SR30N",c("X122023_Condition")]<- "CLP"
# can't find photos b/w 9/23 - 01/24
colony_data[colony_data$ID=="MCAV_59_SR30N",c("X012024_Condition")]<- "Healthy"
colony_data[colony_data$ID=="MCAV_59_SR30N",c("X022024_Condition")]<- "Healthy"
colony_data[colony_data$ID=="MCAV_59_SR30N",condition_columns]

colonies_to_check <- append(colonies_to_check, c('MCAV_59_SR30N','SSID_51_SR30N','SSID_74_SR30N'))

,X052023_Condition,X092023_Condition,X112023_Condition,X122023_Condition,X012024_Condition,X022024_Condition,X042024_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
50,Healthy,CLP,CLP,CLP,Healthy,Healthy,


,X062019_Condition,X052022_Condition,X122022_Condition,X052023_Condition,X092023_Condition,X112023_Condition,X122023_Condition,X012024_Condition,X022024_Condition,X042024_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
50,Healthy,Diseased,Healthy,Healthy,CLP,CLP,CLP,Healthy,Healthy,


In [133]:
# ssid 51 
# 5/22 sample notes: TL/DC very little tissue; no pic of coral but tagged with yellow numberless tag
# not found 12/22 
# 9/23 this is the weird one that says 'not 51 but adjacent'.. don't see a picture 
# 11/23 note says '51/not 51' & '90% CLB; Coral next to 51; Sampled RNA/DNA at bleaching/healthy margin; immune was bleached tissue'
# safe to assume ACTUAL 51 was never sampled after 5/22 and then this is a random coral sampled starting in 2023?
# 01 & 04 2024 sampled 51??
# dont see pic in 2019 or 2024 
## ACTUAL 51: healthy 2019, diseased in 5/22, dead 12/22 and beyond.. also change sample type to diseased tissue in 5/22
## change the tag num for the one sampled in 9/23 to NOT51. make note about how it was newly sampled in 2023 and has no actual tag 
colony_data[colony_data$ID=="SSID_51_SR30N",condition_columns]
colony_data[colony_data$ID=="SSID_51_SR30N",c("X052022_Condition")]<- "Diseased"
colony_data[colony_data$ID=="SSID_51_SR30N",c("X122022_Condition")]<- "Dead"
colony_data[colony_data$ID=="SSID_51_SR30N",filter_condition_columns]<- "Dead"

,X062019_Condition,X052022_Condition,X122022_Condition,X052023_Condition,X092023_Condition,X112023_Condition,X122023_Condition,X012024_Condition,X022024_Condition,X042024_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
42,Healthy,Diseased,Dead,Dead,Dead,Dead,Dead,Dead,Dead,Dead


,X062019_Condition,X052022_Condition,X122022_Condition,X052023_Condition,X092023_Condition,X112023_Condition,X122023_Condition,X012024_Condition,X022024_Condition,X042024_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
42,Healthy,Diseased,Dead,Dead,Dead,Dead,Dead,Dead,Dead,Dead


In [140]:
## change the tag num for the one sampled in 9/23 to NOT51. make note about how it was newly sampled in 2023 and has no actual tag 
#colony_data[grepl("51", colony_data$NewTagNum), ]
colony_data$NewTagNum %>% unique()

## 51/not 51 is just in sample data not colony data 
# ADD to colony data? (will do manually)

[1] "1"         "2"         "3"         "4"         "5"         "6"        
 [7] "7"         "8"         "9"         "10"        "11"        "12"       
[13] "13"        "14"        "15"        "16"        "17"        "24"       
[19] "19"        "20"        "21"        "22"        "23"        "25"       
[25] "27"        "35"        NA          "28"        "29"        "30"       
[31] "31"        "32"        "51"        "52"        "53"        "54"       
[37] "55"        "56"        "57"        "58"        "59"        "60"       
[43] "61"        "62"        "63"        "64"        "65"        "66"       
[49] "67"        "68"        "69"        "70"        "71"        "72"       
[55] "73"        "74"        "75"        "76"        "77"        "78"       
[61] "79"        "80"        "not_found" "18"        "26"        "33"       
[67] "34"        "36"        "37"        "38"        "39"        "43"       
[73] "92"        "93"        "94"        "95"        "96"        "97"       
[79] "98"        "100"       "50"        "49"        "48"        "47"       
[85] "46"        "45"        "44"        "42"        "41"        "40"

In [141]:
# ssid 74
# sampled as diseased in 12/22
## CHANGE to diseased 
colony_data[colony_data$ID=="SSID_74_SR30N",c("X122022_Condition")]<- "Diseased"

In [142]:
mismatched_conditions1222[(mismatched_conditions1222$Transect=="CBC30N"),]

,ID,Transect,lh_Condition_Oct19,lh_Condition_May22,lh_Condition_Dec22,X062019_Condition,X052022_Condition,X122022_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
29,PAST_19_CBC30N,CBC30N,Healthy,Diseased,Dead,Healthy,Diseased,Diseased
66,SSID_3_CBC30N,CBC30N,Healthy,Diseased,Healthy,Healthy,Diseased,Diseased


In [148]:
# past 19
# sampled as healthy since 2019 
# i think it does look diseased in 5/2022 https://drive.google.com/drive/u/0/folders/1ZLEumaeRQxZuQPliu6cyrt9UIuoEsaJl
# definitely not dead - there's like 3 right next to each other so maybe that's why Leah put dead?
## CHANGE to Healthy 12/2022 - https://drive.google.com/drive/u/0/folders/1t2QVNPuMeeFzvaX7h6GEl-oKBJG9_pFn
## CHANGE sample type in 5/22 to diseased_tissue
colony_data[colony_data$ID=="PAST_19_CBC30N",c("X122022_Condition")]<- "Healthy"

# ssid 3
# looks healthy in 12/22 https://drive.google.com/drive/u/0/folders/1t2QVNPuMeeFzvaX7h6GEl-oKBJG9_pFn
## CHANGE to healthy 
colony_data[colony_data$ID=="SSID_3_CBC30N",c("X122022_Condition")]<- "Healthy"

colonies_to_check <- append(colonies_to_check, c('PAST_19_CBC30N','SSID_3_CBC30N'))

In [149]:
mismatched_conditions1222[(mismatched_conditions1222$Transect=="Lagoon"),]

,ID,Transect,lh_Condition_Oct19,lh_Condition_May22,lh_Condition_Dec22,X062019_Condition,X052022_Condition,X122022_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
27,PAST_13_Lagoon,Lagoon,Healthy,Increased Old Mortality,Increased Old Mortality,Healthy,Healthy,Healthy
33,PAST_24_Lagoon,Lagoon,Healthy,Healthy,Healthy,Healthy,Dead,Dead
64,SSID_26_Lagoon,Lagoon,Healthy,Diseased,Diseased,Healthy,Dead,Dead
68,SSID_33_Lagoon,Lagoon,Healthy,Healthy,Healthy,Healthy,Healthy,Diseased
69,SSID_36_Lagoon,Lagoon,Healthy,Diseased,Healthy,Healthy,Diseased,Diseased


In [150]:
# past 13
## LEAVE as healthy (disregard inc old mort)

# past 24 - see above for 5/22 issues
# 5/22 notes say missed sample and 12/22 sampled as healthy: think we labelled as dead and then never returned again
# (old tag 314) https://drive.google.com/drive/u/0/folders/1LeYcwxCL0uu-b3GyRJiC6dyUuUP3hDC7
# 5/22 https://drive.google.com/drive/u/0/folders/1neqfKWg7dLPKk1f4FS6QuN_CXcLy9eOe
## CHANGE to healthy in 5/22 and 12/22 and then...?
## RESOLUTION ABOVE - fixed

# ssid 26 - see above for 5/22 issues
# sampled as diseased in 12/22, no sample in 5/22 
# looks alive but diseased in all 2022, never sampled after (guessing it was presumed dead?)
## CHANGE to diseased in 12/22? https://drive.google.com/drive/u/0/folders/1WPO_drh8oriwWwarB7gQ4g5aGgrd7Qyu
colony_data[colony_data$ID=="SSID_26_Lagoon",c("X122022_Condition")]<- "Diseased"
colony_data[colony_data$ID=="SSID_26_Lagoon",filter_condition_columns] <- "Not_visited"

# ssid 33 
# looks healthy 12/22, always sampled as healthy https://drive.google.com/drive/u/0/folders/1WPO_drh8oriwWwarB7gQ4g5aGgrd7Qyu
## CHANGE to Healthy 
colony_data[colony_data$ID=="SSID_33_Lagoon",c("X122022_Condition")]<- "Healthy"

# ssid 36 
# sampled as diseased 5/22, healthy as 12/22 
# looks healthy 12/22 https://drive.google.com/drive/u/0/folders/1WPO_drh8oriwWwarB7gQ4g5aGgrd7Qyu
## CHANGE to healthy 12/22
colony_data[colony_data$ID=="SSID_36_Lagoon",c("X122022_Condition")]<- "Healthy"

colonies_to_check <- append(colonies_to_check, c('PAST_13_Lagoon','PAST_24_Lagoon','SSID_26_Lagoon','SSID_33_Lagoon','SSID_36_Lagoon'))

In [152]:
colonies_to_check

[[1]]
[1] "MCAV_61_SR30N"

[[2]]
[1] "PAST_56_SR30N"

[[3]]
[1] "PAST_68_SR30N"

[[4]]
[1] "PSTR_62_SR30N"

[[5]]
[1] "MCAV_7_CBC30N"

[[6]]
[1] "SSID_10_CBC30N"

[[7]]
[1] "PAST_13_Lagoon"

[[8]]
[1] "PAST_24_Lagoon"

[[9]]
[1] "PSTR_4_Lagoon"

[[10]]
[1] "SSID_26_Lagoon"

[[11]]
[1] "MCAV_59_SR30N"

[[12]]
[1] "SSID_51_SR30N"

[[13]]
[1] "SSID_74_SR30N"

[[14]]
[1] "PAST_19_CBC30N"

[[15]]
[1] "SSID_3_CBC30N"

[[16]]
[1] "PAST_13_Lagoon"

[[17]]
[1] "PAST_24_Lagoon"

[[18]]
[1] "SSID_26_Lagoon"

[[19]]
[1] "SSID_33_Lagoon"

[[20]]
[1] "SSID_36_Lagoon"

In [154]:
# double check all conditions for all colonies 
colony_data[colony_data$ID %in% colonies_to_check,c("ID",condition_columns)]

,ID,X062019_Condition,X052022_Condition,X122022_Condition,X052023_Condition,X092023_Condition,X112023_Condition,X122023_Condition,X012024_Condition,X022024_Condition,X042024_Condition
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
3,SSID_3_CBC30N,Healthy,Diseased,Healthy,Diseased,"CLP,CLD","CLP,CLB",CLP,CLP,CLB,
7,MCAV_7_CBC30N,Healthy,Healthy,Healthy,Healthy,Healthy,Healthy,Healthy,Healthy,Healthy,
10,SSID_10_CBC30N,Healthy,Diseased,Diseased,Diseased,Dead,Dead,Dead,Dead,Dead,
19,PAST_19_CBC30N,Healthy,Diseased,Healthy,Diseased,Diseased,Diseased,Diseased,Diseased,Diseased,
42,SSID_51_SR30N,Healthy,Diseased,Dead,Dead,Dead,Dead,Dead,Dead,Dead,Dead
47,PAST_56_SR30N,Healthy,Diseased,Healthy,Healthy,Healthy,Healthy,Healthy,Healthy,Healthy,Dead
50,MCAV_59_SR30N,Healthy,Diseased,Healthy,Healthy,CLP,CLP,CLP,Healthy,Healthy,
52,MCAV_61_SR30N,Healthy,Healthy,Dead,Dead,Dead,Dead,Dead,Dead,Dead,
53,PSTR_62_SR30N,Healthy,Healthy,Dead,Dead,Dead,Dead,Dead,Dead,Dead,


In [155]:
filter_condition_columns

[1] "X052023_Condition" "X092023_Condition" "X112023_Condition"
[4] "X122023_Condition" "X012024_Condition" "X022024_Condition"
[7] "X042024_Condition"

In [157]:
colony_data[colony_data$ID=="SSID_3_CBC30N",c("X122023_Condition","X012024_Condition","X022024_Condition")] <- "Healthy"

In [158]:
# healthy in 9/23, not visited 11/23 or 12/23, healthy 01/24 and onward 
colony_data[colony_data$ID=="PAST_19_CBC30N",c("X052023_Condition","X092023_Condition","X012024_Condition","X022024_Condition")]<- "Healthy"
colony_data[colony_data$ID=="PAST_19_CBC30N",c('X112023_Condition','X122023_Condition')]<- "Not_visited"

In [159]:
# disease 9/23, not visited 11/23, 12/23, 1/24, or 2/24
colony_data[colony_data$ID=="SSID_74_SR30N",c("X092023_Condition")] <- "Diseased"
colony_data[colony_data$ID=="SSID_74_SR30N",c('X112023_Condition','X122023_Condition',"X012024_Condition","X022024_Condition")]<- "Not_visited"

In [160]:
# 9/23 sample notes are dc 10% which I agree with from pic, not visited 11/23 or 12/23, healthy 01/2024
colony_data[colony_data$ID=="SSID_36_Lagoon",c("X092023_Condition")] <- "CLD"
colony_data[colony_data$ID=="SSID_36_Lagoon",c("X092023_Percentage")] <- "10%"
colony_data[colony_data$ID=="SSID_36_Lagoon",c("X112023_Condition",'X122023_Condition')] <- "Not_visited"
colony_data[colony_data$ID=="SSID_36_Lagoon",c("X012024_Condition")] <- "Healthy"

In [161]:
# remove added columns (will remove 5/23 condition col manually just in case)
colony_data$Year_tagged <- NULL
colony_data$ID <- NULL

In [162]:
getwd()

[1] "/Users/brookesienkiewicz/Documents/sctld/SCTLD_samples/Sample_Data"

In [164]:
colnames(colony_data)

[1] "Date_InitialTag"          "Transect"                
 [3] "TransectNum"              "OldTagNum"               
 [5] "NewTagNum"                "Species"                 
 [7] "Meter"                    "Meters_90"               
 [9] "Direction"                "Size_Class"              
[11] "Notes_June_2019"          "MaxDiameter"             
[13] "Height"                   "Date_DocumentedDisease"  
[15] "Date_DocumentedMortality" "Notes_May2022"           
[17] "Sept_2023"                "Nov_2023"                
[19] "Dec_2023"                 "Jan_2024"                
[21] "Feb_2024"                 "April_2024"              
[23] "X062019_Condition"        "X062019_Percentage"      
[25] "X052022_Condition"        "X122022_Condition"       
[27] "X052023_Condition"        "X092023_Condition"       
[29] "X092023_Percentage"       "X112023_Condition"       
[31] "X112023_Percentage"       "X122023_Condition"       
[33] "X122023_Percentage"       "X012024_Condition"       
[35] "X012024_Percentage"       "X022024_Condition"       
[37] "X022024_Percentage"       "X042024_Condition"       
[39] "X042024_Percentage"       "COLONIES_TO_LOOK_AT"

In [163]:
# export colony data 
write.csv(colony_data, file = "CBC_ColonyData.csv")